### Flash Card Data To Firebase

In [1]:
#!pip install google-cloud-storage

In [2]:
#!pip install firebase_admin

In [3]:
#!pip install firebase

In [4]:
#!pip install --upgrade pip

In [5]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [6]:
#!pip install pyrebase4

In [7]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [8]:
#import pyrebase
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import shutil

In [9]:
#import firebase_admin
#from firebase_admin import credentials

cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")
app = firebase_admin.initialize_app(cred)

In [10]:
fire_db = firestore.client()

#### Read Data

In [11]:
data_root = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Root_Word_Translate_99.xlsx")
data_root

,sequence,root,root_arabic,root_arabic_frequency,surah,ayah,word rank,surah latin,arabic_simple,arabic,arabic_frequency,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,1,qwl,قول,1722,2,8,4,Al-Baqara,يقول,يَقُولُ,35,say,وہ کہتا ہے,वो कहता है,berkata,বলে,der,говорит:
1,1,qwl,قول,1722,2,11,2,Al-Baqara,قيل,قِيلَ,34,it is said,کہا گیا,कहा जाता है,dikatakan,বলা হয়,dendiği,говорится
2,1,qwl,قول,1722,2,11,8,Al-Baqara,قالوا,قَالُوا,250,They said,انہوں نے کہا,उन्होंने कहा,mereka berkata,তারা বললো,dediler,Сказали они:
3,1,qwl,قول,1722,2,13,2,Al-Baqara,قيل,قِيلَ,34,it is said,کہا گیا,कहा जाता है,dikatakan,বলা হয়,dendiği,говорится
4,1,qwl,قول,1722,2,13,8,Al-Baqara,قالوا,قَالُوا,250,They said,انہوں نے کہا,उन्होंने कहा,mereka berkata,তারা বললো,dediler,Сказали они:
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27372,99,nhr,نهر,113,85,11,11,Al-Burooj,الأنهار,الْأَنْهَارُ,40,the rivers,نہریں,नहरें,sungai-sungai,ঝর্ণাসমূহ,ırmaklar,"реки,"
27373,99,nhr,نهر,113,91,3,1,Ash-Shams,والنهار,وَالنَّهَارِ,13,and the day,اور دن کو,और दिन के,dan siang,ও দিনের,ve gündüzün,"и дня,"
27374,99,nhr,نهر,113,92,2,1,Al-Lail,والنهار,وَالنَّهَارِ,13,and the day,اور دن کو,और दिन के,dan siang,ও দিনের,ve gündüzün,"и дня,"
27375,99,nhr,نهر,113,93,10,4,Ad-Dhuha,تنهر,تَنْهَرْ,1,repel,جھڑکو,आप झिड़कये (उसे),kamu menghardik,তিরস্কার করো,azarlama,отгоняй!


In [12]:
data_word = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Word_Translate_300.xlsx")
data_word.reset_index(inplace=True)
data_word.rename(columns={"index":"sequence"}, inplace=True)
data_word["sequence"] = data_word["sequence"].apply(lambda x: x+1)
data_word

,sequence,arabic_simple,arabic_simple_frequency,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,1,من,2763,from,سے,जो,dari,থেকে,"-den,-dan",из
1,2,الله,2153,Allah,اللہ,अल्लाह,Allah,আল্লাহ,Allah,Аллах
2,3,في,1185,in,میں,ज़मीन में,dalam,মধ্যে,içinde,в
3,4,ما,1013,what,جو,जो,apa,যা,şeyleri,"то, что"
4,5,إن,966,Indeed,بیشک,बेशक,sesungguhnya,নিশ্চয়ই,şüphesiz,"Поистине,"
...,...,...,...,...,...,...,...,...,...,...
300,301,مريم,33,(of) Maryam,مریم,ऐ मरियम,Maryam,মারইয়ামের,Hz. Meryem,"Марьям,"
301,302,بعضهم,33,some of them,ان میں سے بعض,उनके बाज़ को,sebagian mereka,তাদের একে,bir kısmı,одни из них
302,303,ربكما,33,(of) your Lord,اپنے رب کی,अपने रब की,Tuhanmu berdua,তোমাদের উভয়ের রবের,Rabbinizin,Господа вашего (двоих)
303,304,وفي,33,and in,اور میں,और आख़िरत में,dan pada,এবং মধ্যে,ve içinde,и в


In [13]:
data_twogram = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Twogram_Translate_Frequency.xlsx")
data_twogram.reset_index(inplace=True)
data_twogram.rename(columns={"index":"sequence"}, inplace=True)
data_twogram["sequence"] = data_twogram["sequence"].apply(lambda x: x+1)
data_twogram

,sequence,twogram_string,twogram_string_frequency,twogram_english,twogram_turkish,twogram_urdu,twogram_hindi,twogram_indonesian,twogram_bangla,twogram_russian
0,1,إن الله,205,God is,Allah,خدا ہے,भगवान है,Tuhan adalah,উপাস্য নেই,Бог это
1,2,الذين آمنوا,184,who believed,kim inandı,جو ایمان لائے,जो विश्वास करता था,siapa yang percaya?,যারা বিশ্বাস করেছিল,кто верил
2,3,في الأرض,176,in Earth,Dünya'da,زمین میں,धरती में,di Bumi,পৃথিবীতে,на Земле
3,4,الذين كفروا,134,who disbelieved,kim inanmadı,جنہوں نے کفر کیا۔,जिसने अविश्वास किया,siapa yang tidak percaya?,যারা অবিশ্বাস করেছিল,кто не поверил
4,5,السماوات والأرض,133,Heavens and Earth,Gökler ve Dünya,آسمان اور زمین,स्वर्ग और पृथ्वी,Langit dan Bumi,আকাশ ও পৃথিবী,Небеса и Земля
...,...,...,...,...,...,...,...,...,...,...
195,196,من بعدهم,17,after them,onlardan sonra,ان کے بعد,उनके बाद,setelah mereka,তাদের পরে,после них
196,197,ولو شاء,17,if he wants,eğer isterse,اگر وہ چاہتا ہے,यदि वह चाहता है,jika dia mau,যদি সে চায়,если он хочет
197,198,إن كان,17,if,Eğer,اگر,अगर,jika,যদি,если
198,199,رسول الله,17,Messenger of God,Resulullah,خدا کے رسول,भगवान के दूत,Utusan Tuhan,রসূল সা,Посланник Бога


In [14]:
data_threegram = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Threegram_Translate_Frequency.xlsx")
data_threegram.reset_index(inplace=True)
data_threegram.rename(columns={"index":"sequence"}, inplace=True)
data_threegram["sequence"] = data_threegram["sequence"].apply(lambda x: x+1)
data_threegram

,sequence,threegram_string,threegram_string_frequency,threegram_english,threegram_turkish,threegram_urdu,threegram_hindi,threegram_indonesian,threegram_bangla,threegram_russian
0,1,بسم الله الرحمن,114,"in the name of ALLAH, the most mercifull",merhametli olan Allah adıyla,اللہ کے نام سے جو سب سے زیادہ رحم کرنے والا ہے۔,"अल्लाह के नाम पर, सबसे दयालु",dengan menyebut nama ALLAH yang maha penyayang,পরম করুণাময় আল্লাহর নামে,во имя АЛЛАХА Милостивого
1,2,يا أيها الذين,92,O you who,Ey sen kim,اے جو,हे तुम कौन,Wahai kamu siapa,হে আপনি যারা,"О ты, кто"
2,3,أيها الذين آمنوا,89,O you who believe,Ey inananlar,اے ایمان والو!,हे तुम जो विश्वास करते हो,Wahai orang-orang yang beriman,হে ঈমানদারগণ,"О вы, кто верит"
3,4,من دون الله,71,without God,Tanrı olmadan,خدا کے بغیر,भगवान के बिना,tanpa Tuhan,ঈশ্বর ছাড়া,без Бога
4,5,على كل شيء,52,for everything,herşey için,ہر چیز کے لئے,प्रत्येक वस्तु के लिए,untuk semuanya,সব কিছুর জন্য,За все
...,...,...,...,...,...,...,...,...,...,...
195,196,الأرض بعد موتها,8,earth after death,ölümden sonra dünya,موت کے بعد زمین,मृत्यु के बाद पृथ्वी,bumi setelah kematian,মৃত্যুর পরে পৃথিবী,земля после смерти
196,197,إن وعد الله,8,God's promise,Tanrı'nın vaadi,خدا کا وعدہ,भगवान का वादा,janji tuhan,ঈশ্বরের প্রতিশ্রুতি,обещание Бога
197,198,إن الله هو,8,God is,Tanrı,خدا ہے,भगवान है,Tuhan adalah,উপাস্য নেই,Бог это
198,199,إلا أن يشاء,8,except that he wills,onun dilemesi dışında,سوائے اس کے کہ وہ چاہے,सिवाय इसके कि वह चाहता है,kecuali dia berkehendak,ব্যতীত যে সে ইচ্ছা করে,"кроме того, что он хочет"


In [15]:
data_ayah = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Flash Cards/Deployment/Quran_Plain_Ayahs_Frequency.xlsx")
data_ayah.reset_index(inplace=True)
data_ayah.rename(columns={"index":"sequence"}, inplace=True)
data_ayah["sequence"] = data_ayah["sequence"].apply(lambda x: x+1)
data_ayah

,sequence,arabic_sent_tanzil_plain,arabic_sent_frequency
0,1,فَبِأَيِّ آلَاءِ رَبِّكُمَا تُكَذِّبَانِ,31
1,2,وَيْلٌ يَوْمَئِذٍ لِلْمُكَذِّبِينَ,11
2,3,وَإِنَّ رَبَّكَ لَهُوَ الْعَزِيزُ الرَّحِيمُ,8
3,4,فَاتَّقُوا اللَّهَ وَأَطِيعُونِ,8
4,5,بِسْمِ اللَّهِ الرَّحْمَٰنِ الرَّحِيمِ حم,7
...,...,...,...
6050,6051,وَكُلَّ إِنْسَانٍ أَلْزَمْنَاهُ طَائِرَهُ فِي ...,1
6051,6052,وَجَعَلْنَا اللَّيْلَ وَالنَّهَارَ آيَتَيْنِ ف...,1
6052,6053,وَيَدْعُ الْإِنْسَانُ بِالشَّرِّ دُعَاءَهُ بِا...,1
6053,6054,وَأَنَّ الَّذِينَ لَا يُؤْمِنُونَ بِالْآخِرَةِ...,1


#### Sending Data To Firestore

##### Root Data

In [16]:
#data_root = data_root.iloc[0:3150,]  # data divide because of quota
data_root = data_root.drop_duplicates(subset = ['arabic_simple', 'arabic'], keep = 'last').reset_index(drop = True)
data_root

,sequence,root,root_arabic,root_arabic_frequency,surah,ayah,word rank,surah latin,arabic_simple,arabic,arabic_frequency,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,1,qwl,قول,1722,2,26,23,Al-Baqara,فيقولون,فَيَقُولُونَ,1,[thus] they will say,تو وہ کہتے ہیں,तो वो कहते हैं,maka mereka berkata,তারা বলে তখন,derler ki,то они говорят
1,1,qwl,قول,1722,2,102,29,Al-Baqara,يقولا,يَقُولَا,1,they [both] say,وہ دونوں کہتے,वो दोनों कहते,berkata keduanya,দুজনে বলেছে,demedikçe,говорили они двое:
2,1,qwl,قول,1722,2,118,17,Al-Baqara,قولهم,قَوْلِهِمْ,2,their saying,ان کی بات کی,उनकी बात के,ucapan mereka,তাদের কথার,onların sözlerinin,их словам
3,1,qwl,قول,1722,2,263,1,Al-Baqara,قول,قَوْلٌ,1,A word,بات,बात,perkataan,কথা,bir söz (söylemek),Речь
4,1,qwl,قول,1722,3,147,3,Al Imran,قولهم,قَوْلَهُمْ,1,their words,ان کی بات,बात उनकी,perkataan mereka,তাদের কথা,sözleri,"речью их,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6204,99,nhr,نهر,113,73,20,19,Al-Muzzammil,والنهار,وَالنَّهَارَ,12,and the day,اور دن کو,और दिन को,dan siang,ও দিনকে,ve gündüzü,"и день,"
6205,99,nhr,نهر,113,78,11,2,An-Naba',النهار,النَّهَارَ,12,the day,دن کو,दिन को,siang,দিনকে,gündüzü,день
6206,99,nhr,نهر,113,92,2,1,Al-Lail,والنهار,وَالنَّهَارِ,13,and the day,اور دن کو,और दिन के,dan siang,ও দিনের,ve gündüzün,"и дня,"
6207,99,nhr,نهر,113,93,10,4,Ad-Dhuha,تنهر,تَنْهَرْ,1,repel,جھڑکو,आप झिड़कये (उसे),kamu menghardik,তিরস্কার করো,azarlama,отгоняй!


In [17]:
# root
sequence_list = data_root["sequence"].to_list()
root_arabic_list = data_root["root_arabic"].to_list()
root_list = data_root["root"].to_list()
root_arabic_freq_list = data_root["root_arabic_frequency"].to_list()
# word 
arabic_simple_list = data_root["arabic_simple"].to_list()
arabic_list = data_root["arabic"].to_list()
trans_eng_list = data_root["translate_english"].to_list()
trans_urdu_list = data_root["translate_urdu"].to_list()
trans_hindi_list = data_root["translate_hindi"].to_list()
trans_indo_list = data_root["translate_indonesian"].to_list()
trans_bangla_list = data_root["translate_bangla"].to_list()
trans_tur_list = data_root["translate_turkish"].to_list()
trans_rus_list = data_root["translate_russian"].to_list()
#surah_list = data_root["surah"].to_list()
#ayah_list = data_root["ayah"].to_list()
#word_rank_list = data_root["word rank"].to_list()

In [18]:
#firestore_sending_root_data = zip(sequence_list,root_arabic_list,root_list,root_arabic_freq_list,arabic_simple_list,arabic_list,surah_list,ayah_list,word_rank_list,trans_eng_list,trans_urdu_list,trans_hindi_list,trans_indo_list,trans_bangla_list,trans_tur_list,trans_rus_list)
firestore_sending_root_data = zip(sequence_list,root_arabic_list,root_list,root_arabic_freq_list,arabic_simple_list,arabic_list,trans_eng_list,trans_urdu_list,trans_hindi_list,trans_indo_list,trans_bangla_list,trans_tur_list,trans_rus_list)

In [19]:
# Sending Root Data
root_check = 1
data_list = []
data = dict()
data_all_list = []
#for sequence,root_arabic,root,root_arabic_freq,arabic_simple,arabic,surah,ayah,word_rank,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_root_data:
for sequence,root_arabic,root,root_arabic_freq,arabic_simple,arabic,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_root_data:
    if sequence == root_check:
        data_var = {"root_arabic":root_arabic,
            "root":root,
            "root_arabic_freq":root_arabic_freq,
            "arabic_simple":arabic_simple,
            "arabic":arabic,
            #"surah":surah,
            #"ayah":ayah,
            #"word_rank":word_rank,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus
            }
    
        data_list.append(data_var)
        #data["root"] = data_list
    else:
        data_all_list.append(data_list)
        #fire_db.collection("Root_Data").document(f"root{sequence}").set(data)    
        root_check += 1
        data_list = []
        data_var = {"root_arabic":root_arabic,
            "root":root,
            "root_arabic_freq":root_arabic_freq,
            "arabic_simple":arabic_simple,
            "arabic":arabic,
            #"surah":surah,
            #"ayah":ayah,
            #"word_rank":word_rank,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus
            }

        data_list.append(data_var)
    
    #fire_db.collection("Root_Data").document(f"root{sequence}").collection(f"{root}").document(f"{arabic_simple}").set(data)
    #fire_db.collection("Root_Data").document(f"root{sequence}").set(data)
    
    # Not: Eger document ile bitiyorsa data set ile gönderiliyor.
    #fire_db.collection("Root_Data").document(f"root{sequence}").collection("Detail").add(data)  
     
    # Not: Eğer collection ile bitiyorsa data add ile gönderiliyor ve collection dan sonraki document adları rastgele oluşturuluyor.                                                                                             

In [26]:
for i in range(len(data_all_list)):
    data["root"] = data_all_list[i]
    fire_db.collection("Root_Data").document(f"root{i+1}").set(data)

In [22]:
#fire_db.collection(u'cities').document(u'DC').delete()

##### Word Data

In [36]:
data_word

,sequence,arabic_simple,arabic_simple_frequency,translate_english,translate_urdu,translate_hindi,translate_indonesian,translate_bangla,translate_turkish,translate_russian
0,1,من,2763,from,سے,जो,dari,থেকে,"-den,-dan",из
1,2,الله,2153,Allah,اللہ,अल्लाह,Allah,আল্লাহ,Allah,Аллах
2,3,في,1185,in,میں,ज़मीन में,dalam,মধ্যে,içinde,в
3,4,ما,1013,what,جو,जो,apa,যা,şeyleri,"то, что"
4,5,إن,966,Indeed,بیشک,बेशक,sesungguhnya,নিশ্চয়ই,şüphesiz,"Поистине,"
...,...,...,...,...,...,...,...,...,...,...
300,301,مريم,33,(of) Maryam,مریم,ऐ मरियम,Maryam,মারইয়ামের,Hz. Meryem,"Марьям,"
301,302,بعضهم,33,some of them,ان میں سے بعض,उनके बाज़ को,sebagian mereka,তাদের একে,bir kısmı,одни из них
302,303,ربكما,33,(of) your Lord,اپنے رب کی,अपने रब की,Tuhanmu berdua,তোমাদের উভয়ের রবের,Rabbinizin,Господа вашего (двоих)
303,304,وفي,33,and in,اور میں,और आख़िरत में,dan pada,এবং মধ্যে,ve içinde,и в


In [37]:
# word
sequence_list = data_word["sequence"].to_list()
arabic_simple_list = data_word["arabic_simple"].to_list()
arabic_simple_freq_list = data_word["arabic_simple_frequency"].to_list()
trans_eng_list = data_word["translate_english"].to_list()
trans_urdu_list = data_word["translate_urdu"].to_list()
trans_hindi_list = data_word["translate_hindi"].to_list()
trans_indo_list = data_word["translate_indonesian"].to_list()
trans_bangla_list = data_word["translate_bangla"].to_list()
trans_tur_list = data_word["translate_turkish"].to_list()
trans_rus_list = data_word["translate_russian"].to_list()

In [38]:
firestore_sending_word_data = zip(sequence_list,arabic_simple_list,arabic_simple_freq_list,trans_eng_list,trans_urdu_list,trans_hindi_list,trans_indo_list,trans_bangla_list,trans_tur_list,trans_rus_list)

In [39]:
# Sending Word Data
data = dict()
for sequence,arabic_simple,arabic_simple_freq,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_word_data:
    data_var = [{"arabic_simple":arabic_simple,
            "arabic_simple_freq":arabic_simple_freq,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}]
    data["word"] = data_var 
    #fire_db.collection("Word_Data").document(f"word{sequence}").collection(f"{arabic_simple}").document("detail").set(data)
    fire_db.collection("Word_Data").document(f"word{sequence}").set(data)

##### Twogram Data

In [28]:
data_twogram

,sequence,twogram_string,twogram_string_frequency,twogram_english,twogram_turkish,twogram_urdu,twogram_hindi,twogram_indonesian,twogram_bangla,twogram_russian
0,1,إن الله,205,God is,Allah,خدا ہے,भगवान है,Tuhan adalah,উপাস্য নেই,Бог это
1,2,الذين آمنوا,184,who believed,kim inandı,جو ایمان لائے,जो विश्वास करता था,siapa yang percaya?,যারা বিশ্বাস করেছিল,кто верил
2,3,في الأرض,176,in Earth,Dünya'da,زمین میں,धरती में,di Bumi,পৃথিবীতে,на Земле
3,4,الذين كفروا,134,who disbelieved,kim inanmadı,جنہوں نے کفر کیا۔,जिसने अविश्वास किया,siapa yang tidak percaya?,যারা অবিশ্বাস করেছিল,кто не поверил
4,5,السماوات والأرض,133,Heavens and Earth,Gökler ve Dünya,آسمان اور زمین,स्वर्ग और पृथ्वी,Langit dan Bumi,আকাশ ও পৃথিবী,Небеса и Земля
...,...,...,...,...,...,...,...,...,...,...
195,196,من بعدهم,17,after them,onlardan sonra,ان کے بعد,उनके बाद,setelah mereka,তাদের পরে,после них
196,197,ولو شاء,17,if he wants,eğer isterse,اگر وہ چاہتا ہے,यदि वह चाहता है,jika dia mau,যদি সে চায়,если он хочет
197,198,إن كان,17,if,Eğer,اگر,अगर,jika,যদি,если
198,199,رسول الله,17,Messenger of God,Resulullah,خدا کے رسول,भगवान के दूत,Utusan Tuhan,রসূল সা,Посланник Бога


In [29]:
# twogram
sequence_list = data_twogram["sequence"].to_list()
twogram_string_list = data_twogram["twogram_string"].to_list()
twogram_string_freq_list = data_twogram["twogram_string_frequency"].to_list()
twogram_trans_eng_list = data_twogram["twogram_english"].to_list()
twogram_trans_urdu_list = data_twogram["twogram_urdu"].to_list()
twogram_trans_hindi_list = data_twogram["twogram_hindi"].to_list()
twogram_trans_indo_list = data_twogram["twogram_indonesian"].to_list()
twogram_trans_bangla_list = data_twogram["twogram_bangla"].to_list()
twogram_trans_tur_list = data_twogram["twogram_turkish"].to_list()
twogram_trans_rus_list = data_twogram["twogram_russian"].to_list()

In [30]:
firestore_sending_twogram_data = zip(sequence_list,twogram_string_list,twogram_string_freq_list,twogram_trans_eng_list,twogram_trans_urdu_list,twogram_trans_hindi_list,twogram_trans_indo_list,twogram_trans_bangla_list,twogram_trans_tur_list,twogram_trans_rus_list)

In [31]:
# Sending Twogram Data
data = dict()
for sequence,twogram,twogram_freq,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_twogram_data:
    data_var = [{"twogram_simple":twogram,
            "twogram_freq":twogram_freq,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}]
    data["twogram"] = data_var 
    #fire_db.collection("Twogram_Data").document(f"twogram{sequence}").collection(f"{twogram}").document("detail").set(data)
    fire_db.collection("Twogram_Data").document(f"twogram{sequence}").set(data)

##### Threegram Data

In [24]:
data_threegram

,sequence,threegram_string,threegram_string_frequency,threegram_english,threegram_turkish,threegram_urdu,threegram_hindi,threegram_indonesian,threegram_bangla,threegram_russian
0,1,بسم الله الرحمن,114,"in the name of ALLAH, the most mercifull",merhametli olan Allah adıyla,اللہ کے نام سے جو سب سے زیادہ رحم کرنے والا ہے۔,"अल्लाह के नाम पर, सबसे दयालु",dengan menyebut nama ALLAH yang maha penyayang,পরম করুণাময় আল্লাহর নামে,во имя АЛЛАХА Милостивого
1,2,يا أيها الذين,92,O you who,Ey sen kim,اے جو,हे तुम कौन,Wahai kamu siapa,হে আপনি যারা,"О ты, кто"
2,3,أيها الذين آمنوا,89,O you who believe,Ey inananlar,اے ایمان والو!,हे तुम जो विश्वास करते हो,Wahai orang-orang yang beriman,হে ঈমানদারগণ,"О вы, кто верит"
3,4,من دون الله,71,without God,Tanrı olmadan,خدا کے بغیر,भगवान के बिना,tanpa Tuhan,ঈশ্বর ছাড়া,без Бога
4,5,على كل شيء,52,for everything,herşey için,ہر چیز کے لئے,प्रत्येक वस्तु के लिए,untuk semuanya,সব কিছুর জন্য,За все
...,...,...,...,...,...,...,...,...,...,...
195,196,الأرض بعد موتها,8,earth after death,ölümden sonra dünya,موت کے بعد زمین,मृत्यु के बाद पृथ्वी,bumi setelah kematian,মৃত্যুর পরে পৃথিবী,земля после смерти
196,197,إن وعد الله,8,God's promise,Tanrı'nın vaadi,خدا کا وعدہ,भगवान का वादा,janji tuhan,ঈশ্বরের প্রতিশ্রুতি,обещание Бога
197,198,إن الله هو,8,God is,Tanrı,خدا ہے,भगवान है,Tuhan adalah,উপাস্য নেই,Бог это
198,199,إلا أن يشاء,8,except that he wills,onun dilemesi dışında,سوائے اس کے کہ وہ چاہے,सिवाय इसके कि वह चाहता है,kecuali dia berkehendak,ব্যতীত যে সে ইচ্ছা করে,"кроме того, что он хочет"


In [25]:
# threegram
sequence_list = data_threegram["sequence"].to_list()
threegram_string_list = data_threegram["threegram_string"].to_list()
threegram_string_freq_list = data_threegram["threegram_string_frequency"].to_list()
threegram_trans_eng_list = data_threegram["threegram_english"].to_list()
threegram_trans_urdu_list = data_threegram["threegram_urdu"].to_list()
threegram_trans_hindi_list = data_threegram["threegram_hindi"].to_list()
threegram_trans_indo_list = data_threegram["threegram_indonesian"].to_list()
threegram_trans_bangla_list = data_threegram["threegram_bangla"].to_list()
threegram_trans_tur_list = data_threegram["threegram_turkish"].to_list()
threegram_trans_rus_list = data_threegram["threegram_russian"].to_list()

In [26]:
firestore_sending_threegram_data = zip(sequence_list,threegram_string_list,threegram_string_freq_list,threegram_trans_eng_list,threegram_trans_urdu_list,threegram_trans_hindi_list,threegram_trans_indo_list,threegram_trans_bangla_list,threegram_trans_tur_list,threegram_trans_rus_list)

In [27]:
# Sending Threegram Data
data = dict()
for sequence,threegram,threegram_freq,trans_eng,trans_urdu,trans_hindi,trans_indo,trans_bangla,trans_tur,trans_rus in firestore_sending_threegram_data:
    data_var = [{"threegram_simple":threegram,
            "threegram_freq":threegram_freq,
            "translate_eng":trans_eng,
            "translate_urdu":trans_urdu,
            "translate_hindi":trans_hindi,
            "translate_indo":trans_indo,
            "translate_bangla":trans_bangla,
            "translate_tur":trans_tur,
            "translate_rus":trans_rus}]
    data["threegram"] = data_var 
    #fire_db.collection("Threegram_Data").document(f"threegram{sequence}").collection(f"{threegram}").document("detail").set(data)
    fire_db.collection("Threegram_Data").document(f"threegram{sequence}").set(data)

##### Ayah Data

#### Test

In [ ]:
data = {"name":"Ertugrul","surname":"Kurubal"}

In [ ]:
fire_db.collection("Root_Data").document("Person").collection("Detail").add(data)

In [ ]:
fire_db.collection("Root_Data2").document("Person2").set(data)

In [ ]:
# delete firestore
fire_db.collection(u"Root_Data").document(u"Person").delete()

In [ ]:
fire_db.collection(u"Root_Data").document(u"Person").update({})

#### Realtime Database Web App

In [ ]:
import pyrebase  # pyrebase running on colab. Tested in colab
import pandas as pd

In [ ]:
print(pyrebase.__version__)

In [ ]:
firebaseConfig = {
    "apiKey": "xxxxxaSyAKKZn1txxxxx",
    "authDomain": "xxxxxx.firebaseapp.com",
    "databaseURL": "https://flashxxxxx",
    "projectId": "flxxxx",
    "storageBucket": "flxxxxxspot.com",
    "messagingSenderId": "71xxxx",
    "appId": "1xxxxxx2277a217f05b965c",
    "measurementId": "G-Bxxxxxx"
    }  # Web App param from firebase and also added databaseURL from Realtime Database Data URL 

In [ ]:
firebase = pyrebase.initialize_app(firebaseConfig)

In [ ]:
realtime_db = firebase.database()

In [ ]:
data = {"name":"John", "age":25, "adress":["new york","los angeles"]}

In [ ]:
# db.push(data)  # create random name

In [ ]:
#db.child("Test_Data").child("Var").set(data)
realtime_db.child("Test_Data").set(data)

In [ ]:
realtime_db.child("real_test2").child("real_test3").remove()  # delete database data

#### DataFrame Data To Firebase (Bucket, Storage)

##### 1 Way

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [ ]:
firebaseConfig = {
    "apiKey": "xxxxxyAKKZn1tvQy9bxxxxxx",
    "authDomain": "flashxxxxxapp.com",
    "projectId": "flaxxxxx",
    "storageBucket": "flxxxxxx.appspot.com",
    "messagingSenderId": "7xxxxx20981",
    "appId": "1:71xxxxx77a217f05b965c",
    "measurementId": "GxxxxxDJS"
}

In [ ]:
cred   = firebase_admin.credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")
app    = firebase_admin.initialize_app(cred, firebaseConfig)
bucket = storage.bucket(app=app)

In [ ]:
def upload_df(df, data_id):
    """
    Upload a Dataframe as a csv to Firebase Storage
    :return: storage_ref
    """

    # Storage location + extension
    storage_ref = data_id + ".csv"

    # Store locally
    df.to_csv(data_id)

    # Upload to Firebase Storage
    blob = bucket.blob(storage_ref)
    with open(data_id,'rb') as local_file:
        blob.upload_from_file(local_file)

    # Delete locally
    os.remove(data_id)

    return storage_ref

In [ ]:
df_test = pd.read_csv("Test.csv")
df_test

In [ ]:
# upload readed file with func
upload_df(df_test, "bucket_test")

In [ ]:
# upload file directly without read
bucket.blob(blob_name="test4").upload_from_filename("/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Quran/Flash Cards/ReDe/Test.csv")
# blob_name firebase storage name

##### 2 Way

In [ ]:
import os   # pyrebase running on colab. Tested in colab
import pyrebase
import pandas as pd

In [ ]:
firebaseConfig = {
    "apiKey": "xxxxxxxxZn1tvQy9xxxxxx4",
    "authDomain": "xxxxxardxxxxxp.com",
    "databaseURL": "httpsxxxxxc46e-default-rtdxxxxxxxxxxp/",
    "projectId": "flxxxxxxxxx",
    "storageBucket": "flxxxxxxxxt.com",
    "messagingSenderId": "7xxxxxx981",
    "appId": "1:xxxx120xxxxxx05b965c",
    "measurementId": "GxxxxxxxxS",
    "serviceAccount":"/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json"
    }  # Web App param from firebase and added databaseURL from Realtime Database Data URL and also added serviceAccount

In [ ]:
firebase = pyrebase.initialize_app(firebaseConfig)
storage = firebase.storage()

In [ ]:
df = pd.read_csv("/content/Test.csv")
df

In [ ]:
fileName = "FuturePrices.csv"

In [ ]:
# upload file directly without read
storage.child("fileName").put("/media/kurubal/SSD/Data Scientist/Data Science/Data-Scientist/Work/Modern Ways/Quran/Flash Cards/ReDe/Test.csv")